In [91]:
!pip3 -qqq install pytelegrambotapi --upgrade
!pip -qqq install pymorphy2
!pip -qqq install python-Levenshtein

In [93]:
!wget -q https://raw.githubusercontent.com/tesemnikov-av/ParanoidAndroidMarvinBot/master/CORPUS/top250.csv

In [94]:
from datetime import datetime
import Levenshtein
import pandas as pd
import nltk 
import numpy as np
import re
from nltk.stem import wordnet # to perform lemmitization
from sklearn.feature_extraction.text import CountVectorizer # to perform bow
from sklearn.feature_extraction.text import TfidfVectorizer # to perform tfidf
from nltk import pos_tag # for parts of speech
from sklearn.metrics import pairwise_distances # to perfrom cosine similarity
from nltk import word_tokenize # to create tokens
from nltk.corpus import stopwords # for stop words
import telebot
import random
import time
from sklearn.metrics.pairwise import linear_kernel
import pymorphy2
import string

nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

morph = pymorphy2.MorphAnalyzer()
tfidf = TfidfVectorizer()

In [95]:
questions = ['Какой фильм тебе нравится?' , 'Дай мне название какого-нибудь фильма.']
answers = ['Ты продолжай, когда я тебя пойму, я тебе отвечу...' , 'Давай, накидывай, я разберусь...' , 'Тебя родные хоть понимают?']

In [96]:
my_stop_words = stopwords.words('russian')
my_stop_words.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', '–', 'к', 'на', '...'])

In [97]:
def tokenize_ru(file_text):
  tokens = word_tokenize(file_text)
  tokens = [i for i in tokens if (i not in string.punctuation)]
  tokens = [i for i in tokens if (i not in my_stop_words)]
  tokens = [i.replace("«", "").replace("»", "") for i in tokens]
  tokens = [ morph.parse(i)[0].normal_form for i in tokens ]
  return str(tokens).strip('[]').replace("'","").replace(",","")

In [98]:
df_top250 = pd.read_csv('top250.csv', sep='#' , names=['original_title','year', 'country' , 'score' , 'overview','director','screenplay' , 'actors' , 'url'])
df_top250['overview_tokenize'] = df_top250['overview'].map(lambda x: tokenize_ru(str(x)))
df_top250['original_title_lower'] = df_top250['original_title'].map(lambda x: x.lower().strip())
df_top250['creators'] = df_top250[df_top250.columns[5:8]].apply(
    lambda x: ','.join( x.dropna().astype(str)),
    axis=1).apply(lambda x: str(x).replace(" ","").replace(","," "))

In [99]:
df_top250.head(5)

,original_title,year,country,score,overview,director,screenplay,actors,url,overview_tokenize,original_title_lower,creators
0,Побег из Шоушенка,1994,США,9.111,Бухгалтер Энди Дюфрейн обвинён в убийстве собс...,Фрэнк Дарабонт,"Фрэнк Дарабонт, Стивен Кинг","Тим Роббинс, Морган Фриман, Боб Гантон, Уильям...",'https://st.kp.yandex.net/images/film_iphone/i...,бухгалтер энди дюфрейн обвинить убийство собст...,побег из шоушенка,ФрэнкДарабонт ФрэнкДарабонт СтивенКинг ТимРобб...
1,Зеленая миля,1999,США,9.062,Пол Эджкомб — начальник блока смертников в тюр...,Фрэнк Дарабонт,"Фрэнк Дарабонт, Стивен Кинг","Том Хэнкс, Дэвид Морс, Бонни Хант, Майкл Кларк...",'https://st.kp.yandex.net/images/film_iphone/i...,пол эджкомба начальник блок смертник тюрьма х...,зеленая миля,ФрэнкДарабонт ФрэнкДарабонт СтивенКинг ТомХэнк...
2,Форрест Гамп,1994,США,8.913,"От лица главного героя Форреста Гампа, слабоум...",Роберт Земекис,"Эрик Рот, Уинстон Грум","Том Хэнкс, Робин Райт, Салли Филд, Гэри Синиз,...",'https://st.kp.yandex.net/images/film_iphone/i...,от лицо главное герой форрест гампа слабоумный...,форрест гамп,РобертЗемекис ЭрикРот УинстонГрум ТомХэнкс Роб...
3,Список Шиндлера,1993,США,8.817,Фильм рассказывает реальную историю загадочног...,Стивен Спилберг,"Стивен Зеллиан, Томас Кенилли","Лиам Нисон, Бен Кингсли, Рэйф Файнс, Кэролайн ...",'https://st.kp.yandex.net/images/film_iphone/i...,фильм рассказывать реальный история загадочный...,список шиндлера,СтивенСпилберг СтивенЗеллиан ТомасКенилли Лиам...
4,1+1,2011,Франция,8.807,"Пострадав в результате несчастного случая, бог...","Оливье Накаш, Эрик Толедано","Оливье Накаш, Филипп Поццо ди Борго, Эрик Т...","Франсуа Клюзе, Омар Си, Анн Ле Ни, Одри Флеро,...",'https://st.kp.yandex.net/images/film_iphone/i...,пострадать результат несчастный случай богатый...,1+1,ОливьеНакаш ЭрикТоледано ОливьеНакаш ФилиппПоц...


In [101]:
tfidf = TfidfVectorizer(stop_words=my_stop_words)

tfidf_matrix_creators = tfidf.fit_transform(df_top250['creators'])
cosine_sim_creators = linear_kernel(tfidf_matrix_creators, tfidf_matrix_creators)

tfidf_matrix_overview = tfidf.fit_transform(df_top250['overview_tokenize'])
cosine_sim_overview = linear_kernel(tfidf_matrix_overview, tfidf_matrix_overview)
indices = pd.Series(df_top250.index, index=df_top250['original_title_lower']).drop_duplicates()

In [102]:
indices

original_title_lower
побег из шоушенка                          0
зеленая миля                               1
форрест гамп                               2
список шиндлера                            3
1+1                                        4
                                        ... 
крамер против крамера                    245
пираты карибского моря: на краю света    246
иди и смотри                             247
профессионал                             248
холодное лето пятьдесят третьего...      249
Length: 250, dtype: int64

In [103]:
def get_recommendations(title, cosine_sim=cosine_sim_creators ):
    idx = indices[title]

    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    a = df_top250['url'].iloc[movie_indices]
    return str(a.values[random.randint(0,9)].capitalize())[1:-1] # a.values , sim_scores

In [104]:
get_recommendations('криминальное чтиво')

'https://st.kp.yandex.net/images/film_iphone/iphone360_2656.jpg'

In [106]:
df_dialogs = pd.read_excel('dialog_talk_agent.en.ru (4).xlsx', names=['Context','Text Response'])
df_dialogs.ffill(axis = 0,inplace=True)
df_dialogs = df_dialogs.append({'Context' : 'Ты повторяешься' , 'Text Response' : 'Да и ты тоже'} , ignore_index=True)
df_dialogs = df_dialogs.append({'Context' : 'Пойдем пить пиво' , 'Text Response' : 'Да возьми мне пару баночек'} , ignore_index=True)
df_dialogs['lemmatized_text'] = df_dialogs['Context'].map(lambda x: tokenize_ru(str(x)))

In [107]:
x_tfidf=tfidf.fit_transform((df_dialogs['lemmatized_text'])).toarray() 
df_tfidf=pd.DataFrame(x_tfidf,columns=tfidf.get_feature_names())

In [108]:
def chat_tfidf(text):
    lemma=tokenize_ru(text) # calling the function to perform text normalization
    tf=tfidf.transform([lemma]).toarray() # applying tf-idf
    cos=1-pairwise_distances(df_tfidf,tf,metric='cosine') # applying cosine similarity
    index_value=cos.argmax() # getting index value 
    return df_dialogs['Text Response'].loc[index_value]

In [111]:
chat_tfidf('Привет! Как дела?')

'Тогда ладно.'

In [ ]:
LEV_DIST = 4

bot = telebot.TeleBot('1171053774:AAEKARUcqloBdKpZyT9_g4jnIKDcOUJYOKU')
# @ParanoidAndroidMarvinBot

def random_movie():
  return df_top250['original_title'][random.randint(0,len(df_top250['original_title'])-1)]
  
def get_button():
  keyboard1 = telebot.types.ReplyKeyboardMarkup(True, True)
  return keyboard1.row(random_movie().capitalize(),random_movie().capitalize(),random_movie().capitalize())
  del keyboard1

def get_elem(elem):
  return elem[random.randint(0, len(elem)-1)]

def greating():
  hour = int(datetime.strftime(datetime.now()  , '%H')) + 3

  if(hour >= 4 and hour < 12):
     greeting = "Доброе утро!"
  elif (hour >= 12 and hour < 16):
     greeting = "Добрый день!"
  elif (hour >= 16 and hour < 23):
     greeting = "Добрый вечер!"
  return greeting

@bot.message_handler(commands=['start'])
def start_message(message):
  bot.send_message(message.chat.id, greating() )
  bot.send_sticker(message.chat.id, get_elem(stickers_ids) )
  bot.send_message(message.chat.id, get_elem(questions) , parse_mode="markdown" , reply_markup=get_button() ) 

@bot.message_handler(content_types=['text'])
def send_text(message):
    recomended = False

    for film in list(df_top250['original_title_lower']):
      ldis = Levenshtein.distance(message.text.lower(), film)
      if ldis < LEV_DIST:
        recomended = True
        bot.send_photo(message.chat.id, get_recommendations(str(film))) #str(get_recommendations('The Hateful Eight')     )) #    str(get_recommendations('The Hateful Eight')) )
       
    if recomended != True:
   
        #time.sleep(1)
        bot.send_message(message.chat.id, chat_tfidf(message.text.lower()) , reply_markup=get_button())

        if bool(random.choices([True, False], weights=[40, 60])[0]):
          bot.send_sticker(message.chat.id, get_elem(stickers_ids) )

# get stiker id
# @bot.message_handler(content_types=['sticker'])
# def sticker_id(message):
#     print(message.sticker.file_id)


bot.polling()